# Analys av åldersfördelningen bland EU-kandidaterna

av [Anna Norberg](https://twitter.com/annanorberg)

Grunddata kommer från Valmyndigheten och har bearbetats något innan den läses in här. Bland annat har åldern beräknats utifrån personnummer. Åldern gäller den 20 mars 2019.

Resultatet av analysen har använts som underlag till en artikel som skickades ut till Nyhetsbyrån Sirens kunder den 22 mars 2019.

In [45]:
library(tidyr)
library(dplyr)
library(openxlsx)
library(readxl)

**Läs in Excelfilen.**

In [46]:
df1 <- read_excel("data/raw-kandidater.xlsx")

**Hur ser det ut med antal kandidater på kommunnivå?**

In [50]:
kommun <- df1 %>%
group_by(Kommun) %>%
summarize("Yngsta" = min(`Ålder`), 
          "Äldsta" = max(`Ålder`), 
          "Medelålder" = round(mean(`Ålder`), 1), 
          "Antal kandidater" = sum(!is.na(Ålder)))

In [51]:
head(kommun)

Kommun,Yngsta,Äldsta,Medelålder,Antal kandidater
Ale,NA,NA,NA,0
Alingsås,45,54,49.7,3
Alvesta,NA,NA,NA,0
Aneby,NA,NA,NA,0
Arboga,NA,NA,NA,0
Arjeplog,NA,NA,NA,0


**Sortera efter antal kandidater per kommun.**

In [52]:
kommun <- arrange(kommun, desc(`Antal kandidater`))
head(kommun)

Kommun,Yngsta,Äldsta,Medelålder,Antal kandidater
Stockholm,19,85,42.8,48
Göteborg,20,77,44.4,25
Malmö,19,68,39.6,21
Uppsala,22,54,39.5,11
Lund,24,47,31.3,10
Umeå,22,73,40.5,10


**Hur ser åldersfördelningen ut på länsnivå?**

In [53]:
lan <- df1 %>%
group_by(Län) %>% 
summarize("Yngsta" = min(`Ålder`, na.rm = TRUE), 
          "Äldsta" = max(`Ålder`, na.rm = TRUE), 
          "Medelålder" = round(mean(`Ålder`, na.rm = TRUE), 1), 
          "Antal kandidater" = sum(!is.na(Ålder)))

**Sortera efter medelålder.**

In [54]:
lan <- arrange(lan, desc(Medelålder))
head(lan)

Län,Yngsta,Äldsta,Medelålder,Antal kandidater
Ingen svensk adress,57,57,57.0,1
Västernorrlands län,39,65,50.4,10
Dalarnas län,33,63,48.2,12
Östergötlands län,27,75,47.5,15
Jämtlands län,33,57,47.0,8
Örebro län,24,58,46.8,9


**Vilket parti har de äldsta och yngsta kandidaterna?**

In [55]:
parti <- df1 %>% 
group_by(Parti) %>%
summarize("Yngsta" = min(`Ålder`), 
          "Äldsta" = max(`Ålder`), 
          "Medelålder" = round(mean(`Ålder`), 1), 
          "Antal kandidater" = sum(!is.na(Ålder)))

**Sortera efter medelålder.**

In [56]:
parti <- arrange(parti, desc(Medelålder))
head(parti)

Parti,Yngsta,Äldsta,Medelålder,Antal kandidater
SKÅNEPARTIET,75,75,75.0,1
Medborgerlig Samling,39,66,50.5,13
Klassiskt liberala partiet,34,65,50.0,5
Feministiskt initiativ,26,62,47.8,5
Djurens parti,31,64,46.8,5
Kristdemokraterna,18,78,45.7,43


**Hur ser det ut totalt?**

In [57]:
riket <- df1 %>%
summarize("Yngsta" = min(`Ålder`, na.rm = TRUE), 
          "Äldsta" = max(`Ålder`, na.rm = TRUE), 
          "Medelålder" = round(mean(`Ålder`, na.rm = TRUE), 1), 
          "Antal kandidater" = sum(!is.na(Ålder)))
head(riket)

Yngsta,Äldsta,Medelålder,Antal kandidater
18,85,43.5,420


**Sortera rådata efter län och kommun.**

In [58]:
df1 <- arrange(df1, Län, Kommun)

**Gör ny Excelfil med alla tabeller.**

In [62]:
wb<-createWorkbook()

addWorksheet(wb, "Per län")
addWorksheet(wb, "Per kommun")
addWorksheet(wb, "Per parti")
addWorksheet(wb, "Riket")
addWorksheet(wb, "Rådata")

writeData(wb, sheet="Per län", lan, startCol=1, startRow=5, rowNames=FALSE)
writeData(wb, sheet="Per kommun", kommun, startCol=1, startRow=5, rowNames=FALSE)
writeData(wb, sheet="Per parti", parti, startCol=1, startRow=5, rowNames=FALSE)
writeData(wb, sheet="Riket", riket, startCol=1, startRow=5, rowNames=FALSE)
writeData(wb, sheet="Rådata", df1, startCol=1, startRow=1, rowNames=FALSE)

titleStyle <- createStyle(fontSize=15, halign="left", textDecoration="bold", numFmt="@")
titlecells <- expand.grid(row=1:3, col=1)

addStyle(wb, "Per län", rows=titlecells$row, cols=titlecells$col, style=titleStyle)
addStyle(wb, "Per kommun", rows=titlecells$row, cols=titlecells$col, style=titleStyle)
addStyle(wb, "Per parti", rows=titlecells$row, cols=titlecells$col, style=titleStyle)
addStyle(wb, "Riket", rows=titlecells$row, cols=titlecells$col, style=titleStyle)

head1 <- c("EU-kandidaternas medelålder per län", "Källa: Valmyndigheten.", "Sorterat efter medelålder.")
head2 <- c("EU-kandidaternas medelålder per kommun", "Källa: Valmyndigheten.", "Sorterat efter antal kandidater.")
head3 <- c("EU-kandidaternas medelålder per parti", "Källa: Valmyndigheten.", "Sorterat efter medelålder.")
head4 <- c("EU-kandidaternas medelålder", "Källa: Valmyndigheten.")

writeData(wb, sheet="Per län", head1)
writeData(wb, sheet="Per kommun", head2)
writeData(wb, sheet="Per parti", head3)
writeData(wb, sheet="Riket", head4)

headerStyle <- createStyle(fontSize=12, halign="left", textDecoration="bold")
addStyle(wb, 1, headerStyle, rows=5, cols=1:5, gridExpand=TRUE)
addStyle(wb, 2, headerStyle, rows=5, cols=1:5, gridExpand=TRUE)
addStyle(wb, 3, headerStyle, rows=5, cols=1:5, gridExpand=TRUE)
addStyle(wb, 4, headerStyle, rows=5, cols=1:5, gridExpand=TRUE)

nrStyle <- createStyle(numFmt="0")
addStyle(wb, 1, style=nrStyle, cols=5, rows=6:296, gridExpand=TRUE)
addStyle(wb, 2, style=nrStyle, cols=5, rows=6:296, gridExpand=TRUE)
addStyle(wb, 3, style=nrStyle, cols=5, rows=6:296, gridExpand=TRUE)
addStyle(wb, 4, style=nrStyle, cols=5, rows=6:296, gridExpand=TRUE)
addStyle(wb, 5, style=nrStyle, cols=5, rows=6:296, gridExpand=TRUE)

setColWidths(wb, sheet=1, cols=1:5, widths="25", ignoreMergedCells=TRUE)
setColWidths(wb, sheet=2, cols=1:5, widths="25", ignoreMergedCells=TRUE)
setColWidths(wb, sheet=3, cols=1:5, widths="25", ignoreMergedCells=TRUE)
setColWidths(wb, sheet=4, cols=1:6, widths="25", ignoreMergedCells=TRUE)
setColWidths(wb, sheet=5, cols=1:6, widths="25", ignoreMergedCells=TRUE)

saveWorkbook(wb, "output/kandidaterage.xlsx", overwrite=TRUE)